##### Copyright 2022 Los autores de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Composición de algoritmos de aprendizaje

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/composing_learning_algorithms"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">VIEN en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.by/github/tensorflow/federated/blob/v0.33.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.33.0/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/composing_learning_algorithms.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar libreta</a></td>
</table>

## Antes de que empieces

Antes de comenzar, ejecute lo siguiente para asegurarse de que su entorno esté configurado correctamente. Si no ve un saludo, consulte la guía de [instalación](../install.md) para obtener instrucciones. 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from typing import Callable

import tensorflow as tf
import tensorflow_federated as tff

**NOTA** : Se ha verificado que esta colaboración funciona con la [última versión lanzada](https://github.com/tensorflow/federated#compatibility) del paquete pip `tensorflow_federated` , pero el proyecto Tensorflow Federated todavía está en desarrollo preliminar y es posible que no funcione en `main` .

# Composición de algoritmos de aprendizaje

El tutorial Cómo crear [su propio algoritmo de aprendizaje](https://github.com/tensorflow/federated/blob/v0.33.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) federado utilizó el núcleo federado de TFF para implementar directamente una versión del algoritmo de promedio federado (FedAvg).

En este tutorial, utilizará componentes de aprendizaje federado en la API de TFF para crear algoritmos de aprendizaje federado de manera modular, sin tener que volver a implementar todo desde cero.

A los efectos de este tutorial, implementará una variante de FedAvg que emplea el recorte de gradiente a través del entrenamiento local.

## Bloques de construcción de algoritmos de aprendizaje

En un nivel alto, muchos algoritmos de aprendizaje se pueden separar en 4 componentes separados, denominados **bloques de construcción** . Estos son los siguientes:

1. Distribuidor (es decir, comunicación de servidor a cliente)
2. Trabajo del cliente (es decir, cálculo del cliente local)
3. Agregador (es decir, comunicación de cliente a servidor)
4. Finalizador (es decir, cómputo del servidor utilizando salidas de cliente agregadas)

Si bien el [tutorial Construir su propio algoritmo de aprendizaje federado](https://github.com/tensorflow/federated/blob/v0.33.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb) implementó todos estos componentes básicos desde cero, esto a menudo es innecesario. En su lugar, puede reutilizar bloques de construcción de algoritmos similares.

En este caso, para implementar FedAvg con recorte de degradado, solo necesita modificar el componente básico del **trabajo del cliente** . Los bloques restantes pueden ser idénticos a los que se utilizan en FedAvg "vainilla".

# Implementación del trabajo del cliente

Primero, escribamos la lógica TF que realiza el entrenamiento del modelo local con recorte de gradiente. Para simplificar, los gradientes se recortarán con la norma como máximo 1.

## Lógica TF

In [ ]:
@tf.function
def client_update(model: tff.learning.Model,
                  dataset: tf.data.Dataset,
                  server_weights: tff.learning.ModelWeights,
                  client_optimizer: tf.keras.optimizers.Optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = tff.learning.ModelWeights.from_model(model)
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  # Keep track of the number of examples as well.
  num_examples = 0.0
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)
      num_examples += tf.cast(outputs.num_examples, tf.float32)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights.trainable)

    # Compute the gradient norm and clip
    gradient_norm = tf.linalg.global_norm(grads)
    if gradient_norm > 1:
      grads = tf.nest.map_structure(lambda x: x/gradient_norm, grads)

    grads_and_vars = zip(grads, client_weights.trainable)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  # Compute the difference between the server weights and the client weights
  client_update = tf.nest.map_structure(tf.subtract,
                                        client_weights.trainable,
                                        server_weights.trainable)

  return tff.learning.templates.ClientResult(
      update=client_update, update_weight=num_examples)

Hay algunos puntos importantes sobre el código anterior. Primero, realiza un seguimiento de la cantidad de ejemplos vistos, ya que esto constituirá el *peso* de la actualización del cliente (al calcular un promedio entre los clientes).

En segundo lugar, utiliza [`tff.learning.templates.ClientResult`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientResult) para empaquetar la salida. Este tipo de devolución se utiliza para estandarizar los componentes básicos del trabajo del cliente en `tff.learning` .

## Creación de un ClientWorkProcess

Si bien la lógica TF anterior realizará un entrenamiento local con recorte, aún debe envolverse en el código TFF para crear el bloque de construcción necesario.

Específicamente, los 4 bloques de construcción se representan como [`tff.templates.MeasuredProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/MeasuredProcess) . Esto significa que los 4 bloques tienen una función de `initialize` y `next` que se usa para instanciar y ejecutar el cálculo.

Esto permite que cada bloque de construcción realice un seguimiento de su propio **estado** (almacenado en el servidor) según sea necesario para realizar sus operaciones. Si bien no se usará en este tutorial, se puede usar para cosas como el seguimiento de cuántas iteraciones se han producido o el seguimiento de los estados del optimizador.

La lógica de TF del trabajo del cliente generalmente debe envolverse como [`tff.learning.templates.ClientWorkProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/templates/ClientWorkProcess) , que codifica los tipos esperados que entran y salen de la capacitación local del cliente. Puede ser parametrizado por un modelo y optimizador, como se muestra a continuación.

In [ ]:
def build_gradient_clipping_client_work(
    model_fn: Callable[[], tff.learning.Model],
    optimizer_fn: Callable[[], tf.keras.optimizers.Optimizer],
) -> tff.learning.templates.ClientWorkProcess:
  """Creates a client work process that uses gradient clipping."""

  with tf.Graph().as_default():
    # Wrap model construction in a graph to avoid polluting the global context
    # with variables created for this model.
    model = model_fn()
  data_type = tff.SequenceType(model.input_spec)
  model_weights_type = tff.learning.framework.weights_type_from_model(model)

  @tff.federated_computation
  def initialize_fn():
    return tff.federated_value((), tff.SERVER)

  @tff.tf_computation(model_weights_type, data_type)
  def client_update_computation(model_weights, dataset):
    model = model_fn()
    optimizer = optimizer_fn()
    return client_update(model, dataset, model_weights, optimizer)

  @tff.federated_computation(
      initialize_fn.type_signature.result,
      tff.type_at_clients(model_weights_type),
      tff.type_at_clients(data_type)
  )
  def next_fn(state, model_weights, client_dataset):
    client_result = tff.federated_map(
        client_update_computation, (model_weights, client_dataset))
    # Return empty measurements, though a more complete algorithm might
    # measure something here.
    measurements = tff.federated_value((), tff.SERVER)
    return tff.templates.MeasuredProcessOutput(state, client_result,
                                               measurements)
  return tff.learning.templates.ClientWorkProcess(
      initialize_fn, next_fn)

# Componer un algoritmo de aprendizaje

Pongamos el trabajo del cliente anterior en un algoritmo completo. Primero, configuremos nuestros datos y modelo.

## Preparación de los datos de entrada

Cargue y preprocese el conjunto de datos EMNIST incluido en TFF. Para obtener más detalles, consulte el tutorial de [clasificación de imágenes](federated_learning_for_image_classification.ipynb) .

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

Para introducir el conjunto de datos en nuestro modelo, los datos se aplanan y se convierten en tuplas de la forma `(flattened_image_vector, label)` .

Seleccionemos una pequeña cantidad de clientes y apliquemos el preprocesamiento anterior a sus conjuntos de datos.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## Preparando el modelo

Esto utiliza el mismo modelo que en el tutorial de [clasificación de imágenes](federated_learning_for_image_classification.ipynb) . Este modelo (implementado a través `tf.keras` ) tiene una sola capa oculta, seguida de una capa softmax. Para usar este modelo en TFF, el modelo de Keras se envuelve como [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) . Esto nos permite realizar el [pase hacia adelante](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) del modelo dentro de TFF y [extraer los resultados del modelo](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics) . Para obtener más detalles, consulte también el tutorial de [clasificación de imágenes](federated_learning_for_image_classification.ipynb) .

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## Preparando los optimizadores

Al igual que en [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) , aquí hay dos optimizadores: un optimizador de cliente y un optimizador de servidor. Para simplificar, los optimizadores serán SGD con diferentes tasas de aprendizaje.

In [ ]:
client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)

## Definición de los bloques de construcción

Ahora que el bloque de construcción del trabajo del cliente, los datos, el modelo y los optimizadores están configurados, queda por crear los bloques de construcción para el distribuidor, el agregador y el finalizador. Esto se puede hacer simplemente tomando prestados algunos valores predeterminados disponibles en TFF y que son utilizados por FedAvg.

In [ ]:
@tff.tf_computation()
def initial_model_weights_fn():
  return tff.learning.ModelWeights.from_model(model_fn())

model_weights_type = initial_model_weights_fn.type_signature.result

distributor = tff.learning.templates.build_broadcast_process(model_weights_type)
client_work = build_gradient_clipping_client_work(model_fn, client_optimizer_fn)

# TFF aggregators use a factory pattern, which create an aggregator
# based on the output type of the client work. This also uses a float (the number
# of examples) to govern the weight in the average being computed.)
aggregator_factory = tff.aggregators.MeanFactory()
aggregator = aggregator_factory.create(model_weights_type.trainable,
                                       tff.TensorType(tf.float32))
finalizer = tff.learning.templates.build_apply_optimizer_finalizer(
    server_optimizer_fn, model_weights_type)

## Componer los bloques de construcción

Finalmente, puede usar un **compositor** incorporado en TFF para unir los componentes básicos. Este es un compositor relativamente simple, que toma los 4 bloques de construcción anteriores y conecta sus tipos juntos.

In [ ]:
fed_avg_with_clipping = tff.learning.templates.compose_learning_process(
    initial_model_weights_fn,
    distributor,
    client_work,
    aggregator,
    finalizer
)

# Ejecutando el algoritmo

Ahora que el algoritmo está listo, vamos a ejecutarlo. Primero, **inicialice** el algoritmo. El **estado** de este algoritmo tiene un componente para cada bloque de construcción, junto con uno para los *pesos del modelo global* .

In [ ]:
state = fed_avg_with_clipping.initialize()

state.client_work

()

Como era de esperar, el trabajo del cliente tiene un estado vacío (¡recuerde el código de trabajo del cliente anterior!). Sin embargo, otros bloques de construcción pueden tener un estado no vacío. Por ejemplo, el finalizador realiza un seguimiento de cuántas iteraciones se han producido. Dado que `next` aún no se ha ejecutado, tiene un estado de `0` .

In [ ]:
state.finalizer

[0]

Ahora ejecuta una ronda de entrenamiento.

In [ ]:
learning_process_output = fed_avg_with_clipping.next(state, federated_train_data)

La salida de esto ( `tff.learning.templates.LearningProcessOutput` ) tiene una salida `.state` y `.metrics` . Veamos ambos.

In [ ]:
learning_process_output.state.finalizer

[1]

Claramente, el estado del finalizador se ha incrementado en uno, ya que se ha ejecutado una ronda de `.next` .

In [ ]:
learning_process_output.metrics

OrderedDict([('distributor', ()),
             ('client_work', ()),
             ('aggregator',
              OrderedDict([('mean_value', ()), ('mean_weight', ())])),
             ('finalizer', ())])

Si bien las métricas están vacías, para algoritmos más complejos y prácticos generalmente estarán llenas de información útil.

# Conclusión

Al utilizar el marco de creación de bloques/compositores anterior, puede crear algoritmos de aprendizaje completamente nuevos, sin tener que volver a hacer todo desde cero. Sin embargo, este es sólo el punto de partida. Este marco hace que sea mucho más fácil expresar algoritmos como simples modificaciones de FedAvg. Para obtener más algoritmos, consulte [`tff.learning.algorithms`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms) , que contiene algoritmos como [FedProx](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_prox) y [FedAvg con programación de tasa de aprendizaje del cliente](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg_with_optimizer_schedule) . Estas API pueden incluso ayudar a implementaciones de algoritmos completamente nuevos, como [el agrupamiento federado de k-means](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_fed_kmeans) .